In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

In [3]:
import pandas as pd
import pyreadr

### PC and CD45 (super_population) 

In [4]:
cells_dir = "/home/labs/amit/noamsh/data/mm_2023/cells"
spid_PC_cells_path = cells_dir + "/spid_PC_cells.csv"
mars_PC_cells_path = cells_dir + "/mars_PC_cells.csv"

mars_PC_cID = pd.read_csv(mars_PC_cells_path)['Unnamed: 0']
spid_PC_cID = pd.read_csv(spid_PC_cells_path)['Unnamed: 0']

# mars_PC_cID
all_PC = pd.concat([mars_PC_cID, spid_PC_cID]).reset_index(drop=True)

spid_CD45_cells_path = cells_dir + "/spid_CD45_cells.csv"
mars_CD45_cells_path = cells_dir + "/mars_CD45_cells.csv"

mars_CD45_cID = pd.read_csv(mars_CD45_cells_path)['Unnamed: 0']
spid_CD45_cID = pd.read_csv(spid_CD45_cells_path)['Unnamed: 0']

all_CD45 = pd.concat([mars_CD45_cID, spid_CD45_cID]).reset_index(drop=True)

super_pop = "super_Population"
super_pop_df = pd.concat([pd.DataFrame(data={"cID": all_PC, super_pop: "PC"}),
                           pd.DataFrame(data={"cID": all_CD45, super_pop: "CD45"})]).reset_index(drop=True)
super_pop_df["cID"] = super_pop_df["cID"].astype(str)
super_pop_df

,cID,super_Population
0,W555665,PC
1,W555666,PC
2,W555667,PC
3,W555668,PC
4,W555669,PC
...,...,...
433440,W5217361,CD45
433441,W5217363,CD45
433442,W5217364,CD45
433443,W5217365,CD45


### within PC - healthy/malignant

In [18]:
spid_PC_malignant_cells_path = cells_dir + "/spid_PC_malingnant_cells.csv"
mars_PC_malignant_cells_path = cells_dir + "/mars_PC_malingnant_cells.csv"

spid_PC_malignant_cID = pd.read_csv(spid_PC_malignant_cells_path)['Unnamed: 0']
mars_PC_malignant_cID = pd.read_csv(mars_PC_malignant_cells_path)['Unnamed: 0']

all_PC_malignant = pd.concat([mars_PC_malignant_cID, spid_PC_malignant_cID]).reset_index(drop=True)
malignant_df = pd.DataFrame(all_PC_malignant)
malignant_df["Populations"] = "Malignant"
malignant_df = malignant_df.rename(columns={'Unnamed: 0': "cID"})
malignant_df

,cID,Populations
0,WMC2765088,Malignant
1,WMC2765337,Malignant
2,WMC2765328,Malignant
3,WMC2765092,Malignant
4,WMC2765331,Malignant
...,...,...
94593,W3848916,Malignant
94594,W3848795,Malignant
94595,W3849086,Malignant
94596,W3848932,Malignant


### TME - CD45 annoations

In [28]:
shuangyi_MM_dir = '/home/labs/amit/shuangyi/Project_MM_2023/'
spid_label_path = shuangyi_MM_dir + "/ann_SPID_20231110_CD45.Rds"
mars_label_path = shuangyi_MM_dir + "/ann_MARS_20231110_CD45.Rds"

mars_result = pyreadr.read_r(mars_label_path)
spid_result = pyreadr.read_r(spid_label_path)

df_mars_labels = mars_result[None] 
df_spid_labels = spid_result[None] 

label_col = "Populations"
annot_labels = pd.concat([df_mars_labels[[label_col, "cID"]], df_spid_labels[[label_col, "cID"]]]).reset_index(drop=True)
annot_labels["cID"] = annot_labels["cID"].astype(str)
annot_labels

,Populations,cID
0,T_Naive,W811826
1,T_Naive,WMC3225438
2,T_Naive,WMC339832
3,T_Naive,WMC487931
4,T_Naive,WMC2789363
...,...,...
182919,T_Effector_GZMB,W4408336
182920,T_Effector_GZMB,W3437437
182921,T_Effector_GZMB,W4126978
182922,T_Effector_GZMB,W3904314


In [29]:
print(f"""
annot - {len(annot_labels["cID"])},
CD45 cells - {len(all_CD45)}, 
PC cells - {len(all_PC)},
annot-CD45 intersection - {len(set(annot_labels["cID"]).intersection(all_CD45.values))},
annot-PC intersection - {len(set(annot_labels["cID"]).intersection(all_PC.values))})
""")


annot - 182924,
CD45 cells - 245022, 
PC cells - 188423,
annot-CD45 intersection - 182293,
annot-PC intersection - 625)



In [32]:
super_pop_df

,cID,super_Population
0,W555665,PC
1,W555666,PC
2,W555667,PC
3,W555668,PC
4,W555669,PC
...,...,...
433440,W5217361,CD45
433441,W5217363,CD45
433442,W5217364,CD45
433443,W5217365,CD45


In [46]:
all_labels = super_pop_df.merge(annot_labels, on="cID", how="left", validate="1:1")
all_labels = all_labels.set_index("cID")
all_labels["Populations"].update(malignant_df.set_index("cID")["Populations"])
all_labels["Populations"].value_counts()
# all_labels["super_Population"].value_counts()

Populations
Malignant          94597
Mo                 45825
T_Naive            29892
T_Effector         19835
T_Effector_GZMB    19410
DC                 11705
B                  10798
Mo_CD16            10423
Erythrocytes        6231
NK                  5974
B_Pro               4204
UN                  4067
Mf                  4056
pDC                 2327
Mo_Pro              2061
Mast                1568
DC_IRF8             1390
Neu_Pro             1175
Mega                 802
Fibro                759
PC                   354
Name: count, dtype: int64

## Add health PC annotation

In [51]:
healthy_PC_mask = (all_labels["super_Population"] == "PC") & (all_labels["Populations"] != "Malignant")
all_labels["Populations"][healthy_PC_mask] = "Healthy"
all_labels["Populations"].value_counts()

Populations
Malignant          94597
Healthy            93881
Mo                 45800
T_Naive            29811
T_Effector         19744
T_Effector_GZMB    19401
DC                 11683
B                  10768
Mo_CD16            10418
Erythrocytes        6178
NK                  5973
B_Pro               4197
Mf                  4056
UN                  3891
pDC                 2287
Mo_Pro              2059
Mast                1554
DC_IRF8             1390
Neu_Pro             1162
Mega                 795
Fibro                758
PC                   354
Name: count, dtype: int64

In [52]:
all_labels[all_labels["super_Population"]=="CD45"].value_counts()

super_Population  Populations    
CD45              Mo                 45800
                  T_Naive            29811
                  T_Effector         19744
                  T_Effector_GZMB    19401
                  DC                 11683
                  B                  10768
                  Mo_CD16            10418
                  Erythrocytes        6178
                  NK                  5973
                  B_Pro               4197
                  Mf                  4056
                  UN                  3891
                  pDC                 2287
                  Mo_Pro              2059
                  Mast                1554
                  DC_IRF8             1390
                  Neu_Pro             1162
                  Mega                 795
                  Fibro                758
                  PC                   354
                  Malignant             55
Name: count, dtype: int64

In [53]:
all_labels[all_labels["super_Population"]=="PC"].value_counts()

super_Population  Populations
PC                Malignant      94542
                  Healthy        93881
Name: count, dtype: int64

In [54]:
all_annotation_path = Path(cells_dir, "cells_annotation_20240410.csv")
all_labels.to_csv(all_annotation_path)

## check

In [6]:
all_annotation_path = Path(cells_dir, "cells_annotation_20240410.csv")
pd.read_csv(all_annotation_path)

,cID,super_Population,Populations
0,W555665,PC,Healthy
1,W555666,PC,Healthy
2,W555667,PC,Healthy
3,W555668,PC,Healthy
4,W555669,PC,Healthy
...,...,...,...
433440,W5217361,CD45,NaN
433441,W5217363,CD45,NaN
433442,W5217364,CD45,NaN
433443,W5217365,CD45,NaN
